In [ ]:
# Install required dependencies
! pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken langchain pypdf faiss-cpu

In [1]:
# Setup openAI connections

import keys

embed_key = keys.embed_key
embed_endpoint = "https://raid-openai-e27bcf212.openai.azure.com/"

gpt_key = keys.gpt_key
gpt_endpoint = "https://raid-ses-openai.openai.azure.com/"

In [ ]:
# some logic here to better split the documents instead of just by page??

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

def load_docs(filepath):
    loader = DirectoryLoader(filepath, glob='**/*.pdf', loader_cls=PyPDFLoader)

    docs = loader.load()
    
    return docs

def vector_load(docs, key, endpoint):
    
    embedding_model = OpenAIEmbeddings(
    openai_api_type="azure",
    openai_api_key=key, 
    openai_api_base=endpoint,
    openai_api_version="2023-05-15",
    deployment="text-embedding-ada-002"
    )
    
    # need to build some logic here for checking the database - if exists then just add if not, create
    
    db = FAISS.from_documents(docs, embedding_model)
    
    return db
    

In [135]:
docs = load_docs('./data/')
db = vector_load(docs, embed_key, embed_endpoint)

In [3]:
db = FAISS.load_local("dbstore", OpenAIEmbeddings(
    openai_api_type="azure",
    openai_api_key=embed_key, 
    openai_api_base=embed_endpoint,
    openai_api_version="2023-05-15",
    deployment="text-embedding-ada-002"
    ))

In [13]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory

# initiate llm
llm = AzureChatOpenAI(openai_api_type="azure", 
                      openai_api_version="2023-05-15", 
                      openai_api_base=gpt_endpoint, 
                      openai_api_key=gpt_key, 
                      deployment_name="raidGPT", 
                      temperature=0.0)

retriever = db.as_retriever(search_kwargs = {"k": 10})

memory = ConversationSummaryBufferMemory(llm=llm,
                                   memory_key="chat_history", 
                                   input_key="question", 
                                   output_key="answer", 
                                   return_messages=True)

qa = ConversationalRetrievalChain.from_llm(llm=llm, 
                                           retriever=retriever, 
                                           return_source_documents=True, 
                                           memory = memory)

In [5]:
result = qa({'question' : "What is a FOEL check?"})

In [9]:
print(result["source_documents"][0].metadata["source"])

data\PC-21 Employment Manual - Chapt 10 Straight _ Level AL16 Mar 2023.pdf


## Testing custom prompt

In [5]:
from langchain.prompts.prompt import PromptTemplate

In [6]:
custom_template = """
You are a bot designed to answer military pilot trainees' questions from various flying handbooks and rulebooks. Use the context provided below to answer their questions. If you don't know the answer, just say that you don't know, don't try to make up an answer. 

{context}

Additionally, this was the chat history of your conversation with the user.
{chat_history}

Question: {question}

"""

PROMPT = PromptTemplate.from_template(template=custom_template)

In [14]:
qa = ConversationalRetrievalChain.from_llm(llm=llm, 
                                           retriever=retriever, 
                                           return_source_documents=True, 
                                           memory = memory,
                                           combine_docs_chain_kwargs={"prompt" : PROMPT})

In [15]:
qa({"question": "engine flameout from flight idle"})

{'question': 'engine flameout from flight idle',
 'chat_history': [HumanMessage(content='engine flameout from flight idle', additional_kwargs={}, example=False),
  AIMessage(content='Answer: According to the handbook, an engine flameout from flight idle takes approximately 16-17 seconds. This is based on a technical query sent. If a flameout occurs, the pilot should execute the correct Critical Action Procedures (CAPs) and prioritize turning towards a suitable airfield. The pilot should also check the Inter-Turbine Temperature (ITT) and Gas Generator speed (NG) for signs of relight. If the engine fails to relight, the Engine Pre-Start and Auto Start CAPs should be executed immediately.', additional_kwargs={}, example=False)],
 'answer': 'Answer: According to the handbook, an engine flameout from flight idle takes approximately 16-17 seconds. This is based on a technical query sent. If a flameout occurs, the pilot should execute the correct Critical Action Procedures (CAPs) and prioriti

In [9]:
qa({'question' : "How long does it take the engine to flameout from idle?"})["answer"]

'Answer: The engine takes approximately 16-17 seconds to flameout from flight idle.'

In [16]:
qa({'question' : "What about from 50% power setting?"})

{'question': 'What about from 50% power setting?',
 'chat_history': [HumanMessage(content='engine flameout from flight idle', additional_kwargs={}, example=False),
  AIMessage(content='Answer: According to the handbook, an engine flameout from flight idle takes approximately 16-17 seconds. This is based on a technical query sent. If a flameout occurs, the pilot should execute the correct Critical Action Procedures (CAPs) and prioritize turning towards a suitable airfield. The pilot should also check the Inter-Turbine Temperature (ITT) and Gas Generator speed (NG) for signs of relight. If the engine fails to relight, the Engine Pre-Start and Auto Start CAPs should be executed immediately.', additional_kwargs={}, example=False),
  HumanMessage(content='What about from 50% power setting?', additional_kwargs={}, example=False),
  AIMessage(content='Answer: According to the handbook, an engine flameout from a 50% power setting (TQ) takes approximately 10-11 seconds. This is based on a techn

## Versioning Control Tests

## Alternate embedding models

In [27]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

def load_docs(filepath):
    loader = DirectoryLoader(filepath, glob='**/*.pdf', loader_cls=PyPDFLoader)

    docs = loader.load()
    
    return docs

from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

def vector_load(docs, key, endpoint):
    
    embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="Represent the question for retrieving supporting documents: "
)
    
    # need to build some logic here for checking the database - if exists then just add if not, create
    
    db = FAISS.from_documents(docs, embedding_model)
    
    return db
   

In [ ]:
# Was not able to finish because it was running for 15 minutes but still not done transforming yet
docs = load_docs('./data/')
db_bge = vector_load(docs, embed_key, embed_endpoint)